In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, message="Resolving lazy import graphs")
import time
import igraph as ig
import random
import netdecom as nd
from sage.graphs.all import *

import importlib.util 
import sys 
import os 
so_path = "./libdecom_h.so"
module_name = "decom_h"  
spec = importlib.util.spec_from_file_location(module_name, so_path) 
decom_h = importlib.util.module_from_spec(spec) 
sys.modules[module_name] = decom_h 
spec.loader.exec_module(decom_h) 

In [ ]:
def generate_connected_chordal_igraph(n, k):
    G_sage = graphs.RandomChordalGraph(n, algorithm='growing', k=k)
    
    comps = G_sage.connected_components(sort=True)
    x = len(comps)
    

    if x > 1:
        reps = [c[0] for c in comps]
        edges_to_add = list(zip(reps[:-1], reps[1:]))
        G_sage.add_edges(edges_to_add)
    

    edges = [(u, v) for u, v, _ in G_sage.edges()]
    G_ig = ig.Graph()
    G_ig.add_vertices(n)  
    G_ig.add_edges(edges)
    
    return G_ig



In [ ]:
param_list = [
    (250, 4), (250, 40), 
    (500, 8), (500, 93),
    (750, 11), (750, 155),
    (1000, 15), (1000, 230)
]

rep = 50

for (n, k) in param_list:
    print(f"Starting to process parameter set: n={n}, k={k}, repetitions={rep}")


    cmsa_time = 0.0
    shar_time = 0.0
    econ = 0.0

    for i in range(rep):

        G_ig = generate_connected_chordal_igraph(n, k)
        econ += G_ig.ecount()
        r = sorted(random.sample(range(n), 20))
        
        start_time = time.time() 
        result1 =  decom_h.find_convex_hull(G_ig, r,  method="cmsa")
        cmsa_time += time.time() - start_time

   
        start_time = time.time()
        result2 =  decom_h.SAHR(G_ig,r)
        shar_time += time.time() - start_time

        # Result consistency check
        if set(result1) != set(result2):
            print(f"❌ Error: Parameters (n={n}, k={k}), inconsistent results detected at iteration {i+1}!")
            break

else:
    # Executed only if no break occurred
    avg_edges = econ / rep
    print(f"Average number of edges: {avg_edges:.2f}")
    print(f"Total CMSA runtime: {cmsa_time:.4f} seconds, average runtime: {cmsa_time/rep:.6f} seconds per iteration")
    print(f"Total SHAR runtime: {shar_time:.4f} seconds, average runtime: {shar_time/rep:.6f} seconds per iteration")

